# 데이터 분할, 불균형 해소

머신러닝을 통해 분석 모델을 적용하기 전 수집된 데이터가 분석 모델을 적용하는데 적합하도록 분할하고 데이터의 비율을 균일하게 조정하는 고급 전처리 기법

## #01. 준비작업

### [1] 주요 개념 정리

#### 1) 독립변수($x$)

- 결과를 예측하는데 필요한 조건값
- 일반적인 $y = ax + b$ 방정식에서 $x$에 해당함
- `○○이 ■■에 주는 영향`에서 `○○`에 해당
- `설명변수`라고 부르기도 함

#### 2) 종속변수($y$)

- 머신러닝이 예측하고자 하는 결과값
- 일반적인 $y = ax + b$ 방정식에서 $y$에 해당함
- `○○이 ■■에 주는 영향`에서 `■■`에 해당
- `반응변수`라고 부르기도 함

> 추후 뒤에서 좀 더 자세하게 설명합니다.

### [2] 패키지 참조

`imblearn` 패키지의 설치가 필요하다.

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

from pandas import read_excel
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

### [3] 샘플 데이터 가져오기

In [ ]:
origin = read_excel('https://data.hossam.kr/edadata/gradeuate.xlsx')
origin

## #02. 데이터 분할

### [1] 독립변수, 종속변수 분리하기

In [ ]:
x = origin.drop(['합격여부'], axis=1)
y = origin['합격여부']
x.shape, y.shape

### [2] 훈련데이터와 검증데이터 분할하기

| 구분 | 설명 |
|---|---|
| 훈련데이터 | 머신러닝이 학습에 활용하기 위한 데이터 |
| 검증데이터 | 머신러닝의 학습 결과가 올바른지 검증하는데 사용되는 데이터 |

일반적으로 `7(훈련):3(검증)`으로 분할한다.

```python
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=int, random_state=int)
```

| 파라미터 | 설명 |
|---|---|
| x | 독립변수 |
| y | 종속변수 |
| test_size | 검증데이터의 비율 (`0~1`) |
| random_state | 랜덤시드 값 |

#### 데이터 분할하기

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

#### 분할된 훈련데이터의 종속변수 비율 확인

In [ ]:
y_train.value_counts()

## #03. 데이터 불균형 해소

데이터를 훈련데이터와 검증데이터로 나눈 후 훈련 데이터에 대해서만 수행해야 한다.

### [1] Under Sampling 방식 - Random Under Sampler

많은 비율을 차지하는 다수 집단에서 일부만 샘플링하는 방식

소수 집단의 데이터가 어느 정도 확보되었다고 여겨질 때, 다수 집단의 데이터를 줄여서 균형을 맞춘다.

다수 집단의 유의미한 데이터를 손실할 수 있다는 단점이 있다.

#### `sampling_strategy` 파라미터

| 값 | 설명 |
|--|--|
| `majority` | 다수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님`- 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리 |

In [ ]:
undersampler = RandomUnderSampler(sampling_strategy="majority", random_state=777)
x_under, y_under = undersampler.fit_resample(x_train, y_train)
print(x_under.shape)

y_under.value_counts()

### [2] Over Sampling - Random Over Sampler

소수 집단에서 복원 추출을 수행하는 방법.

언더 샘플링처럼 데이터 중 일부를 취하는 것은 아니기 때문에 데이터 손실은 발생하지 않지만, 동일한 데이터를 여러번 학습 데이터에 포함시키므로 학습 정확도는 높지만 과적합 리스크가 크다.

#### `sampling_strategy` 파라미터

| 값 | 설명 |
|--|--|
| `minority` | 소수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님`- 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리 |

In [ ]:
oversampler = RandomOverSampler(sampling_strategy="minority", random_state=777)
x_over, y_over = oversampler.fit_resample(x_train, y_train)
print(x_over.shape, y_over.shape)

y_over.value_counts()

### [3] Over Sampling - SMOTE

소수 집단의 데이터를 바탕으로 새로운 데이터를 생성.

단순히 소수 집단의 데이터를 복원 추출하는 것이 아니라 소수 집단 데이터를 분석해 어떤 특징이 있는지 살피고 그와 유사한 패턴을 갖는 가짜 데이터를 생성한다.

##### `sampling_strategy` 파라미터

| 값 | 설명 |
|--|--|
| `minority` | 소수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님`- 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리 |

혹은 실수 타입으로 설정할 경우 샘플 수의 비율을 의미

##### `k_neighbors` 파라미터 (int)

합성 샘플을 생성하는데 사용할 샘플의 가장 가까운 이웃 수 (기본값=5)

In [ ]:
smote_sampler = SMOTE(sampling_strategy="minority", k_neighbors=3, random_state=777)
x_sm, y_sm = smote_sampler.fit_resample(x_train, y_train)
print(x_sm.shape, y_sm.shape)

y_sm.value_counts()